<a href="https://colab.research.google.com/github/harshdeepsinghhanspal/Multidocument_RAG_ChromaDB_Groq_LLAMA3.1/blob/main/Multidocument_RAG_ChromaDB_Groq_LLAMA_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [2]:
!pip install python-docx unstructured[docs] docx2txt

In [3]:
!apt-get install poppler-utils
!apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [4]:
import os

from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [5]:
GROQ_API_KEY = "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"

In [6]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [7]:
#loader = DirectoryLoader("data/", glob="./*.pdf", loader_cls=UnstructuredPDFLoader)
from langchain.document_loaders import DirectoryLoader, Docx2txtLoader

loader = DirectoryLoader("data/", glob="*.docx", loader_cls=Docx2txtLoader)
documents = loader.load()

In [8]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [9]:
persist_directory = "doc_db"

In [10]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)

In [12]:
retriever = vectorstore.as_retriever()

In [13]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    max_tokens=500
)

In [14]:
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="Use only the following document excerpts:\n{context}\n\n"
             "Question: {question}\n"
             "Answer strictly based on the excerpts. If not found, say 'Not available in the document'."
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}
)

In [15]:
query = "What does the document say about PID?"
response = qa_chain.invoke({"query":query})

In [16]:
print(response)

{'query': 'What does the document say about PID?', 'result': 'The document says the following about PID:\n\n- A PID controller is a system used to make things work smoothly and accurately. It adjusts things like speed, temperature, or position to reach and maintain a desired target.\n- It works by balancing three actions: P (Proportional), I (Integral), and D (Derivative).\n- P (Proportional) looks at the current error and adjusts based on how big the error is.\n- I (Integral) looks at past errors and tries to fix them.\n- D (Derivative) looks at how quickly the error is changing and predicts future errors to prevent overshooting or sudden changes.\n- By tuning the P, I, and D values, you can make the system respond faster, smoother, or more stable, depending on what you need.\n- The PID controller equation is: Output=Kp⋅e(t)+Ki⋅∫e(t)\u2009dt+Kd⋅de(t)dt\n- PID is suitable for simple systems, but for complex systems or systems with uncertainties, other control techniques may be needed.\

In [17]:
print(response["result"])

The document says the following about PID:

- A PID controller is a system used to make things work smoothly and accurately. It adjusts things like speed, temperature, or position to reach and maintain a desired target.
- It works by balancing three actions: P (Proportional), I (Integral), and D (Derivative).
- P (Proportional) looks at the current error and adjusts based on how big the error is.
- I (Integral) looks at past errors and tries to fix them.
- D (Derivative) looks at how quickly the error is changing and predicts future errors to prevent overshooting or sudden changes.
- By tuning the P, I, and D values, you can make the system respond faster, smoother, or more stable, depending on what you need.
- The PID controller equation is: Output=Kp⋅e(t)+Ki⋅∫e(t) dt+Kd⋅de(t)dt
- PID is suitable for simple systems, but for complex systems or systems with uncertainties, other control techniques may be needed.
- PID variants and enhancements include auto-tuning PID, cascade PID, feedfo

In [18]:
query = "What does the document say about Neural Network ContRol?"
response = qa_chain.invoke({"query":query})
print(response["result"])

According to the document, Neural Network Control works by using machine learning to learn the control strategy by observing the system's behavior. It is better suited for highly non-linear systems or when a precise model is unavailable. Examples of its use include robotics and complex industrial systems.


In [19]:
query = "Tell me about State SpAcE ContRol"
response = qa_chain.invoke({"query":query})
print(response["result"])

How it works: Uses a mathematical model of the system in state-space form and optimizes control actions to minimize a cost function.

When it's better: For multi-variable systems with complex interactions.

Example: Aerospace systems, advanced robotics.


In [20]:
query = "Price of Swift Dzire"
response = qa_chain.invoke({"query":query})
print(response["result"])

Not available in the document.
